The school year profile `csvs` have been downloaded from the [Chicago Data Portal](https://data.cityofchicago.org/).   

## School Year Profiles

The first source of data are the school year profiles:

  - [2016-2017 Profile](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/8i6r-et8s)
  - [2017-2018 Profile](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/w4qj-h7bg)
  - [2018-2019 Profile](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/kh4r-387c)

Files should be downloaded and placed in the `data/chicago_data_portal_csv_files` folder.

There are slight differences in the csv files which require quick preprocessing steps.  These preprocessing steps are packaged in the `src/preprocessing` folder 

In [1]:
# Imports to ensure modules import correctly. 

import os, sys

# Set absolute path to the root folder of the directory
full_path = os.getcwd()
home_folder = 'CPS_GradRate_Analysis'
root = full_path.split(home_folder)[0] + home_folder + '/'
sys.path.append(root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from src.preprocessing.preprocessing import years, paths
from src.preprocessing.preprocessing import create_sp_path_dictionary, import_multiple_sy_profiles

2015-2016 missing


In [105]:
# Load available csvs into a dictionary of dataframes
sp_paths = create_sp_path_dictionary(years[:-1], paths)
df_dict = import_multiple_sy_profiles(sp_paths)

In [106]:
df_dict['2017-2018'].head()

,School_ID,Legacy_Unit_ID,Finance_ID,Short_Name,Long_Name,Primary_Category,Is_High_School,Is_Middle_School,Is_Elementary_School,Is_Pre_School,Summary,Administrator_Title,Administrator,Secondary_Contact_Title,Secondary_Contact,Phone,Fax,CPS_School_Profile,Website,Facebook,Twitter,Youtube,Pinterest,Attendance_Boundaries,Grades_Offered_All,Grades_Offered,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Statistics_Description,Demographic_Description,Dress_Code,PreK_School_Day,Kindergarten_School_Day,School_Hours,Freshman_Start_End_Time,After_School_Hours,Earliest_Drop_Off_Time,Classroom_Languages,Bilingual_Services,Refugee_Services,Title_1_Eligible,PreSchool_Inclusive,Preschool_Instructional,Significantly_Modified,Hard_Of_Hearing,Visual_Impairments,Transportation_Bus,Transportation_El,Transportation_Metra,Average_ACT_School,Mean_ACT,College_Enrollment_Rate_School,College_Enrollment_Rate_Mean,Graduation_Rate_School,Graduation_Rate_Mean,Overall_Rating,Rating_Status,Rating_Statement,Classification_Description,School_Year,Third_Contact_Title,Third_Contact_Name,Fourth_Contact_Title,Fourth_Contact_Name,Fifth_Contact_Title,Fifth_Contact_Name,Sixth_Contact_Title,Sixth_Contact_Name,Seventh_Contact_Title,Seventh_Contact_Name,Network,Is_GoCPS_Participant,Is_GoCPS_PreK,Is_GoCPS_Elementary,Is_GoCPS_High_School,Open_For_Enrollment_Date,Closed_For_Enrollment_Date,School_Latitude,School_Longitude,Location
0,610521,7000,29391,DAVIS M,Sir Miles Davis Magnet Elementary Academy,ES,N,Y,Y,N,Miles Davis Magnet Academy provides each stude...,Principal,Cheryl Elaine Armstrong-Belt,Other,Ms. Kimberly Simon,7.735359e+09,7.735359e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.milesdavismagnet.org,NaN,NaN,NaN,NaN,N,"K,1,2,3,4,5,6,7,8",K-8,237,227,45,10,216,20,0,0,0,0,0,1,0,0,There are 237 students enrolled at DAVIS M. 9...,The largest demographic at DAVIS M is Black. ...,Y,NaN,Full Day,07:45 AM-02:45 PM,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Level 1,GOOD STANDING,"This school received a Level 1 rating, which i...",Schools that specialize in a specific subject ...,School Year 2017-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Network 11,Y,N,Y,N,07/01/2008 12:00:00 AM,NaN,41.771221,-87.666530,"6740 S PAULINA ST\nChicago, Illinois 60636\n(4..."
1,609750,1750,49051,SIMPSON HS,Simpson Academy HS for Young Women,HS,Y,Y,Y,N,``,Principal,Mrs.Sherita D Carter-King,Guidance Counselor,Veronica Falls,7.735348e+09,7.735348e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://www.simpsonacademy.org/,http://www.facebook.com/SAYWHS,NaN,NaN,NaN,N,"6,7,8,9,10,11,12",6-12,34,25,3,3,25,9,0,0,0,0,0,0,0,0,There are 34 students enrolled at SIMPSON HS. ...,The largest demographic at SIMPSON HS is Black...,N,NaN,NaN,8:00 AM - 3:00 PM,NaN,NaN,7:30 AM,Spanish,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Pink,NaN,NaN,NaN,33.3,59.8,23.1,77.5,Level 2+,GOOD STANDING,"This school received a Level 2+ rating, which ...",Schools that have their own processes for enro...,School Year 2017-2018,Office,Rita Somen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Network 15,N,N,N,N,09/01/2004 12:00:00 AM,NaN,41.864365,-87.668722,"1321 S PAULINA ST\nChicago, Illinois 60608\n(4..."
2,610386,1123,67021,PEACE AND EDUCATION HS,Peace & Education Coalition HS,HS,Y,N,N,N,The Peace and Education Coalition Accelerated ...,Principal,Brigitte Lee Swenson,Assistant Principal/Second Chance Campus,Lyssette Guzman,7.735359e+09,7.735359e+09,http://cps.edu/Schools/Pages/school.aspx?Schoo...,http://peaceandeducationschools.org,https://www.facebook.com/Peace-and-Education-C...,NaN,NaN,NaN,N,"10,11,12",10-12,94,58,16,13,29,62,2,0,0,0,0,1,0,0,There are 94 stud

# Isolate important columns



In [116]:
from src.preprocessing.preprocessing import isolate_important_columns

In [128]:
important = {year: isolate_important_columns(df_dict[year]) for year in df_dict}
important['2016-2017']

,School_ID,Graduation_Rate_School,Student_Count_Total,Student_Count_Low_Income,Student_Count_Special_Ed,Student_Count_English_Learners,Student_Count_Black,Student_Count_Hispanic,Student_Count_White,Student_Count_Asian,Student_Count_Native_American,Student_Count_Other_Ethnicity,Student_Count_Asian_Pacific_Islander,Student_Count_Multi,Student_Count_Hawaiian_Pacific_Islander,Student_Count_Ethnicity_Not_Available,Is_High_School,Dress_Code,Classroom_Languages,Transportation_El
0,610163,NaN,232,37,90,27,1,39,175,16,0,0,0,0,1,0,N,N,NaN,NaN
1,610558,NaN,900,788,153,57,459,420,7,2,5,0,0,6,1,0,Y,Y,Spanish,NaN
2,609750,21.4,38,37,6,2,28,8,2,0,0,0,0,0,0,0,Y,N,Spanish,Pink
3,610571,NaN,341,320,57,31,187,148,4,0,1,0,0,1,0,0,Y,Y,Spanish,Blue
4,610123,NaN,311,279,78,13,283,26,1,0,1,0,0,0,0,0,N,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,610317,NaN,454,401,36,190,11,425,6,7,4,0,0,0,0,1,N,Y,Spanish for Heritage Speakers,NaN
657,609749,97.8,1062,470,61,7,78,316,311,209,5,0,3,46,2,92,Y,N,"Chinese, French, Japanese, Latin, Spanish",NaN
658,400172,NaN,494,474,110,148,13,460,9,4,2,0,0,5,0,1,Y,Y,"Spanish, Spanish for Heritage Speakers",NaN
659,609720,95.9,4447,2191,198,27,351,1813,1567,469,18,0,6,118,12,93,Y,Y,"Arabic, Chinese, French, German, Italian, Japa...","Blue, Brown"


In [126]:
for year in df_dict:
    df_dict[year][["School_ID", "Graduation_Rate_School", "Student_Count_Total","Student_Count_Low_Income",
"Student_Count_Special_Ed","Student_Count_English_Learners",
"Student_Count_Black","Student_Count_Hispanic","Student_Count_White",
"Student_Count_Asian","Student_Count_Native_American","Student_Count_Other_Ethnicity",
"Student_Count_Asian_Pacific_Islander","Student_Count_Multi","Student_Count_Hawaiian_Pacific_Islander",                                                           
"Student_Count_Ethnicity_Not_Available", 'Is_High_School', "Dress_Code", "Classroom_Languages","Transportation_El"]]

In [97]:
target_columns = ["School_ID","Administrator","Grades_Offered_All",
"Student_Count_Total","Student_Count_Low_Income",
"Student_Count_Special_Ed","Student_Count_English_Learners",
"Student_Count_Black","Student_Count_Hispanic","Student_Count_White",
"Student_Count_Asian","Student_Count_Native_American","Student_Count_Other_Ethnicity",
"Student_Count_Asian_Pacific_Islander","Student_Count_Multi","Student_Count_Hawaiian_Pacific_Islander",                                                           
"Student_Count_Ethnicity_Not_Available","ADA_Accessible","Dress_Code",
"Classroom_Languages","Transportation_El","Graduation_Rate_School",
"School_Year","Zip"]

df_dict['2018-2019'][target_columns]

KeyError: "['ADA_Accessible'] not in index"

In [76]:
df_dict['2018-2019'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 95 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   School_ID                                660 non-null    int64  
 1   Legacy_Unit_ID                           660 non-null    int64  
 2   Finance_ID                               660 non-null    int64  
 3   Short_Name                               660 non-null    object 
 4   Long_Name                                660 non-null    object 
 5   Primary_Category                         660 non-null    object 
 6   Is_High_School                           660 non-null    bool   
 7   Is_Middle_School                         660 non-null    bool   
 8   Is_Elementary_School                     660 non-null    bool   
 9   Is_Pre_School                            660 non-null    bool   
 10  Summary                                  646 non-n

For each available year in the school profiles, I will separate out high schools from middle schools, since I will be moving towards using graduation rates for supervised learning.


In [74]:
from src.preprocessing.preprocessing import isolate_high_schools

In [77]:
df_hs = {year: isolate_high_schools(df_dict[year]) for year in df_dict}

0      HS
2      HS
5      HS
9      HS
13     HS
       ..
651    HS
652    HS
655    HS
656    HS
658    HS
Name: Primary_Category, Length: 183, dtype: object

In [ ]:
# Interesting: primary category would be a good feature to change to Primary_Is_High_School.  
# This would give a signal of whether a school is a specifically a high school.
df_hs['2018-2019']['Primary_Category']